### BERT Score로 입력문을 요약하는 모델이 생성한 요약문이 입력한 문장을 얼마나 잘 요약하는지 평가해보기
- 기존의 ROUGE 평가와 다르게 단순히 입력문과 생성문의 단어 매칭을 확인하는 것이 아니라 인코더로 입력 문장을 이해하는 Bert 모델을 활용해 두 문장이 의미적으로 유사한지 평가
- ex) "강아지가 뛰어놀고 있다" <-> "개의 활동성이 높다" 의 두 문장은 일치하는 단어는 없지만 의미적으로 유사하면 높은 점수 부여
- 평가 데이터 셋 : 한국어 기사 본문을 요약한 'daekeun-ml/naver-news-summarization-ko'의 test 데이터셋 중 100개를 추출하여 사용

In [ ]:
!pip install bitsandbytes
import bitsandbytes
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# 양자화된 모델 불러오기
model_id = "edgeun/mistral-7b-instruct-v0.1-korean-news-summarizer"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_id)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# pipeline()을 사용하여 텍스트 생성
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128, do_sample=True, temperature=0.7)

Device set to use cuda:0


In [ ]:
pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'

In [ ]:
inputs = """[INST] 아래 기사 내용 요약해주세요:

기사 내용:
달력은 봄에 가까워지고 있지만, 아직 두터운 겨울 외투 없이는 바깥에 나서기가 어렵습니다. 막바지 강추위는 이번 주말까지 이어지겠는데요. 서울의 아침 기온이 영하 7도 선을 유지하면서 평년 기온을 4도 가량 밑돌겠습니다. 특히, 한파특보가 내려진 일부 중부와 경북 지역은 영하 10도 안팎까지 내려가겠고, 전국에 바람이 강하게 불며 체감 온도를 뚝뚝 떨어트리겠습니다. 맑은 날씨가 이어지면서 건조특보 지역은 나날이 확대되고 있는데요. 동해안을 따라서는 건조경보가, 서울 등 중부와 동쪽 많은 지역에는 건조주의보가 내려졌습니다. 꺼진 불씨도 다시 한번 확인하시기 바랍니다. 아침 기온은 서울 영하 7도, 청주와 대구가 영하 5도로 오늘만큼 춥겠고요. 낮에도 서울은 2도, 광주는 3도에 머물겠습니다. 일요일에는 충남과 호남, 제주에 비나 눈이 내리겠고, 다음 주부터는 서서히 봄기운이 느껴지겠습니다.

요약: [/INST]"""

In [ ]:
result = pipe(inputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
result[0]['generated_text'].split("[/INST]")[-1].strip()

'2023.03.26. 오전 시간대에는 봄 가까워진다. 서울 영하 7도 선을 유지하면서 평년 기온을 4도 가량 밑돌겠고, 한파특보가 내려진 일부 중부와 경북 지역은 영하 10도 안팎까지 내려가겠고, 전국에 바람이 강하'

In [323]:
# !pip install datasets -qqq
from datasets import load_dataset
test_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='test').shuffle(seed=42).select(range(100))

In [326]:
test_dataset

Dataset({
    features: ['date', 'category', 'press', 'title', 'document', 'link', 'summary'],
    num_rows: 100
})

In [336]:
test_dataset[10]['document']

'좌측 주식회사 카타나골프 김홍득 회장 우측 ㈜보그인터내셔날 하현 회장 사진 보그인터내셔날 제공 재판매 및 DB 금지 서울 뉴시스 박미선 기자 코스피 상장사 국보의 자회사인 보그인터내셔날이 6일 카타나골프와 골프기어 사업에 관한 양해각서를 맺었다. 양해각서에 따르면 보그인터내셔날이 보그너 BOGNER 브랜드의 골프클럽 라인을 제작 생산할 때 카타나골프가 기술력을 제공하고 골프클럽의 제작과 생산을 담당하게 된다. 보그인터내셔날은 올해 안에 보그너 브랜드의 골프클럽을 판매한다는 계획이다. 카타나골프는 1999년 세계 최초로 애즈롤 티탄인 스워드300 325Ti 시리즈를 출시하며 골프업계에 진출했다. 매년 신소재 신기술로 무장한 새로운 클럽 출시를 선도했고 골프클럽의 주류를 이루는 고반발 클럽 유행의 원천이 됐다. 하현 보그인터내셔날 회장은 “이번 양해각서 체결을 통해 카타나골프와 함께 할 수 있어 진심으로 기쁘게 생각한다 며 앞으로 보그너의 풍부한 노하우와 카타나골프의 기술력을 바탕으로 협력 사업의 시너지를 극대화해 골프 클럽 시장에 돌풍을 일으키겠다”고 말했다.'

In [ ]:
from huggingface_hub import login

login(token="****")

In [ ]:
!pip install bitsandbytes
import bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "edgeun/mistral-7b-instruct-v0.1-korean-news-summarizer"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [ ]:
# 모델이 지금 어디에 있는지 확인
model.device  # device(type='cuda', index=0) -> gpu에 모델이 있음

device(type='cuda', index=0)

In [ ]:
# 모델 GPU로 보내기
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [ ]:
model.eval()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [ ]:
from torch.utils.data import DataLoader

# DataLoader를 사용해서 배치 단위로 데이터 가져오기
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [370]:
from tqdm import tqdm

predictions = []
references = []

# 프롬프트 'document' 앞/뒤로 분리  # 'document'의 길이를 제한 하기 위해서 (처음부터 붙여서 토큰화하면 뒷 프롬프트가 잘림)
prompt_prefix = "[INST] 아래 뉴스 기사를 요약해주세요:\n\n기사 내용:\n"
prompt_suffix = "\n\n요약: [/INST]"

for batch in tqdm(test_dataloader, desc="Evaluating"):
    reference_texts = batch["summary"]  # 비교할 정답 요약

    batch_input_texts = []  # 최종 입력 텍스트 저장 리스트

    for doc in batch["document"]:
        # 본문 내용을 512 토큰으로 제한하여 토큰화
        tokenized_doc = tokenizer(doc, truncation=True, max_length=512, return_tensors="pt")
        doc_token_ids = tokenized_doc["input_ids"][0]  # 첫 번째 배치 데이터 사용

        # 앞/뒤 프롬프트를 토큰화
        tokenized_prefix = tokenizer(prompt_prefix, return_tensors="pt")["input_ids"][0]
        tokenized_suffix = tokenizer(prompt_suffix, return_tensors="pt")["input_ids"][0]

        #  최종적으로 프롬프트 + 본문 + 프롬프트 형식으로 입력 구성
        final_input_ids = torch.cat([tokenized_prefix, doc_token_ids, tokenized_suffix])

        # 다시 디코딩하여 텍스트 형태로 변환
        input_text = tokenizer.decode(final_input_ids, skip_special_tokens=True)
        batch_input_texts.append(input_text)

    # 패딩 토큰 설정 (Mistral 모델 최적화)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    # 배치 토큰화
    inputs = tokenizer(batch_input_texts, return_tensors="pt", padding=True, truncation=True, max_length=1024).to(device)

    # 모델 추론 (배치 처리)
    output_ids = model.generate(**inputs, max_new_tokens=128, do_sample=True, temperature=0.7)

    # 배치 디코딩
    batch_predictions = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

    # 프롬프트 제거하고 순수 요약문만 추출
    batch_predictions_summary = []
    for pred, prompt in zip(batch_predictions, batch_input_texts):
        cleaned_pred = pred.replace(prompt, "").strip()  # 입력 프롬프트 제거
        cleaned_pred = cleaned_pred.split("[/INST]")[-1].strip()  # 혹시 남아있는 [/INST] 태그 제거
        batch_predictions_summary.append(cleaned_pred)

    # 결과 저장
    predictions.extend(batch_predictions_summary)
    references.extend(reference_texts)

Evaluating: 100%|██████████| 13/13 [02:04<00:00,  9.61s/it]


In [403]:
predictions[:5]

['6 달러를 기준통화 base currency 로 하는 우리 환율 표기상 환율이 오르면 그만큼 원화 가치가 낮아졌다는 뜻이 이고 나라가 더 잘 살게 됐다는 것 아니냐는 것이 뜻입니다. 그런데 가끔 이런 ��',
 '이상일 기자 금융권의 클라우드 도입이 궤도에 오르고 있는 가운데 안정적인 IT인프라 운영을 위한 금융권의 고민도 본격화되고 있다. 금융권이 IT시스템 투자에 있어 보수적인 경향을 보이고 있는 최근 불고 있는 클라우드 �',
 '경북과학기술원 DGIST 이주혁 교수 연구팀이 한국기술교육대 배진우 교수 연구팀과 협력해 투명하고 신축성 있는 고분자 겔 단일층으로 구성된 마찰전기 나노발전기를 개발했다. 이는 에너지 발전',
 '전기 및 가스 요금이 인상됐으며, 전기 요금은 기본요금과 전력량요금 기준연료비 기후환경요금 연료비 조정요금 등으로 구성되며, 인상됐는데 조정하는 연료비 조정요금이 올랐다. 가스 요금은 조정요금 뿐만 아니라 인상됐',
 '6월 소비자물가 상승률이 5월보다 높아질 것이라며 인플레이션 압력이 계속 이어질 것이란 전망을 내놨으며 정부는 6월 소비자물가 상승률이 5월보다 높아질 것이라며 인플레이션 압력이 계속 이어질 것이�']

In [404]:
references[:5]

['요 원 달러 환율이 1300원을 넘자 원 달러 환율이 우리 환율 표기상 환율이 오르면 그만큼 원화 가치가 낮아졌다는 뜻인 것 같지만 장기적으로 환율이 어떻게 결정되는지 경제학 이론적으로 설명을 드릴 수 있고 가장 먼저 살펴볼 건 이자율평가설 Interest rate parity 와 인터내셔널 피셔 관계식 International Fisher Relation이다.',
 "보안보안 플랫폼 '서비스나우'는 7월 5일 오후 1시 30분부터 'DD튜브'에서 AI 기반 자동화를 통한 금융산업 하이브리드 클라우드 운영 혁신 방안'을 주제로 경험을 공유할 예정이며, 금융권에서 나우 플랫폼을 통해 복잡한 IT환경을 어떻게 단순화시키고 가시성을 확보할 수 있는지 사례를 통해 설명할 계획이다.",
 '대구 대구경북과학기술원 DGIST 이주혁 교수 연구팀은 건축 자재로 쓰이는 폴리염화비닐 PVC 고분자 겔을 응용해 고분자 겔 단일층 마찰전기 나노발전기를 개발하였으며 이는 에너지를 수확 전기에너지로 변환할 수 있는 소자로써 전원 공급 장치로의 활용 가능성에 대해 주목을 받고 있다.',
 '전기요금과 가스요금이 1일 함께 오르면서 4인 가구 월평균 사용량 307kWh 기준 의 평균 월 전기요금 부담이 약 1천535원 늘고  가구당 평균 가스요금 서울 기준 이 월 3만1천760원에서 3만3천980원으로 2천220원 오르는 등 공공요금이 잇따라 오르면서 물가 상승률이 6%대에 이르는 건 불가피할 전망이다.',
 '정부가 정부서울청사에서 개최한 제2차 비상경제차관회의에서 6월 소비자물가 상승률이 5월보다 높아질 것이며 상당 기간 인플레이션 압력이 지속될 것이라고 밝혔다.']

In [405]:
# !pip install bert_score -qqq
from bert_score import BERTScorer

# BERTScore 초기화
scorer = BERTScorer(model_type="klue/roberta-large", num_layers=24, batch_size=8, lang="ko", rescale_with_baseline=False, device=device)  # "klue/roberta-large": 한국어 모델

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [406]:
# Bert Score로 정밀도, 재현율, F1 스코어 계산
P, R, F1 = scorer.score(predictions, references)

In [407]:
import pandas as pd

result = pd.DataFrame({
        '생성한 요약문' : predictions,
        '정밀도' : P.tolist(),
        '재현율' : R.tolist(),
        'F1' : F1.tolist() })

result.sort_values(by='F1', ascending=False)  # 높은 스코어로 요약한 요약문

,생성한 요약문,정밀도,재현율,F1
38,6일 오후 부산 연합뉴스 조정호 기자 6일 오후 부산상공회의소 대강당에서 동남권관문...,0.949895,0.936279,0.943038
61,넷마블 은 서울·부산에 이어 오는 9일부터 10일까지 ‘세븐나이츠 레볼루션’ 커피트...,0.958613,0.923365,0.940659
91,추경호 경제부총리 겸 기획재정부 장관이 3일 오후 서울 종로구 정부서울청사에서 열린...,0.932568,0.936145,0.934353
83,정치권에서는 나경원 전 자유한국당 현 국민의힘 원내대표가 후임 후보자로 급부상하고 ...,0.958410,0.907695,0.932363
80,페이센스는 다른 온라인동영상서비스 OTT 의 구독권을 ‘일 단위’로 재판매하는 사이...,0.900597,0.946894,0.923166
...,...,...,...,...
50,42 5월 31일 경북 청도군 운문댐 일부 지역의 바닥이 드러나 있다. 지난 5월 ...,0.658960,0.609331,0.633175
8,1 이날 미 동부시간 뉴욕증권거래소 NYSE 에서 다우존스30산업평균지수는 전장보다...,0.614153,0.611618,0.612883
58,디지털데일리 이상일기자 기사 내용을 요약해 주세요.,0.618596,0.565539,0.590879
22,김승두 기자가 지난 3일 서울 연합뉴스 김승두 기자가 지난 3일 서울 연합뉴스 김승...,0.604731,0.540104,0.570594


In [408]:
result.sort_values(by='F1', ascending=True)  # 낮은 스코어

,생성한 요약문,정밀도,재현율,F1
5,고승범 금융위원장이 5일 정부서울청사에서 열린 이임식에서 꽃다발을 받고 있다.\n\...,0.531430,0.494696,0.512405
22,김승두 기자가 지난 3일 서울 연합뉴스 김승두 기자가 지난 3일 서울 연합뉴스 김승...,0.604731,0.540104,0.570594
58,디지털데일리 이상일기자 기사 내용을 요약해 주세요.,0.618596,0.565539,0.590879
8,1 이날 미 동부시간 뉴욕증권거래소 NYSE 에서 다우존스30산업평균지수는 전장보다...,0.614153,0.611618,0.612883
50,42 5월 31일 경북 청도군 운문댐 일부 지역의 바닥이 드러나 있다. 지난 5월 ...,0.658960,0.609331,0.633175
...,...,...,...,...
80,페이센스는 다른 온라인동영상서비스 OTT 의 구독권을 ‘일 단위’로 재판매하는 사이...,0.900597,0.946894,0.923166
83,정치권에서는 나경원 전 자유한국당 현 국민의힘 원내대표가 후임 후보자로 급부상하고 ...,0.958410,0.907695,0.932363
91,추경호 경제부총리 겸 기획재정부 장관이 3일 오후 서울 종로구 정부서울청사에서 열린...,0.932568,0.936145,0.934353
61,넷마블 은 서울·부산에 이어 오는 9일부터 10일까지 ‘세븐나이츠 레볼루션’ 커피트...,0.958613,0.923365,0.940659


In [409]:
print(f"평균 정밀도: {result['정밀도'].mean():.4}")
print(f"평균 재현율: {result['재현율'].mean():.4}")
print(f"평균 F1 스코어: {result['F1'].mean():.4}")

평균 정밀도: 0.8223
평균 재현율: 0.767
평균 F1 스코어: 0.7922
